In [18]:
import numpy as np
def F1(t):
    return 0.063*(t**3)-5.284*(t**2)+4.887*t+412+np.random.normal(0,1)

# Ax=b

In [19]:
A=np.zeros((1000,5))
b=np.zeros((1000,1))
for t in range(0,1000):
    b[t] = F1(t)
    A[t,0] = t**4
    A[t,1] = t**3
    A[t,2] = t**2
    A[t,3] = t
    A[t,4] = 1
x=np.linalg.lstsq(A,b)[0]
print(x)

[[-3.94621938e-12]
 [ 6.30000079e-02]
 [-5.28400485e+00]
 [ 4.88803759e+00]
 [ 4.11867553e+02]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  # Remove the CWD from sys.path while we load stuff.


# non linear
有t 跟F2(t) 求ABCD<br>
但是呢 沒法用linear<br>

In [20]:
def F2(t,A,B,C,D):
    return A*(t**B)+C*np.cos(D*t)+np.random.normal(0,1,t.shape)

In [30]:
T=np.random.random((1000,1))*100  #隨機生成T
b2=  F2(T,0.6,1.2,100,-0.4)      #正解

# GA
每個有10個基因<br>
pool中必有正解<br>
你要找出來

先亂數產10000<br>
算energy= 誤差<br>


In [31]:
def gene2coef(gene):
    A = (np.sum(2**np.arange(10)*gene[0:10])-511)/100
                #2**長度為9vecto #長度為9gene
    #二進制轉十進制 512 ~-511
    B = (np.sum(2**np.arange(10)*gene[10:20])-511)/100
    C = (np.sum(2**np.arange(10)*gene[20:30])-511)
    D = (np.sum(2**np.arange(10)*gene[30:40])-511)/100
    return A,B,C,D

# GA
窮舉很難 2\*40 <br>
原始10000人 殺掉剩一定比例<br>
交配產生後代 有些會突變 反覆30次<br>
有機會產生出正解

In [42]:
N=10000
G=30
survive_rate = 0.05 #只有5%活下
mutation_rate = 0.001 #突變率 (大:增加嘗試 小:穩定 爸媽好小孩普遍好)
survive= round(N*survive_rate)
mutation = round(N*40*mutation_rate) #每人有40基因

In [43]:
pop = np.random.randint(0,2,(N,40))
#原始人 N*40個亂數產的0/1
fit = np.zeros((N,1))

In [ ]:
for generation in range(G):
    print(generation)
    #先算這10000人有多適合活在這個世界上 越小越好!!
    for i in range(N):
        A,B,C,D=gene2coef(pop[i,:])
        fit[i]=np.mean(abs(F2(T,A,B,C,D)-b2))
    #殺人
    sort_fit=np.argsort(fit[:,0]) #排序 小的在前
                        #取index
    pop=pop[sort_fit,:]
    for i in range(survive,N):
        dad_id=np.random.randint(0,survive)
        mom_id=np.random.randint(0,survive)
        #父母不可以自體繁殖
        while(dad_id==mom_id):
            mom_id=np.random.randint(0,survive)
        #基因砟傳
        mask = np.random.randint(0,2,[1,40])
        son = pop[mom_id,:].copy()
        dad = pop[dad_id,:]
        son[mask[0,:]==1] = dad[mask[0,:]==1]
        pop[i,:]=son
    #突變
    for i in range(mutation):
        #挑人
        m=np.random.randint(survive,N)
        #挑基因
        n=np.random.randint(0,40)
        #基因突變 0>1  1>0
        pop[m,n]=1-pop[m,n]
for i in range(N):
        A,B,C,D=gene2coef(pop[i,:])
        fit[i]=np.mean(abs(F2(T,A,B,C,D)-b2))
sort_fit=np.argsort(fit[:,0])
pop=pop[sort_fit,:]
#最最最近的答案
A,B,C,D=gene2coef(pop[0,:])

In [45]:
A,B,C,D

(0.6, 1.2, 100, -0.4)